In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth, col

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/04 16:49:54 WARN Utils: Your hostname, HANDABALDEEP resolves to a loopback address: 127.0.1.1; using 172.31.99.153 instead (on interface eth0)
23/05/04 16:49:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/04 16:49:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_repos = spark.read.parquet("sample_data/sample_repos_big_query_dump_1000.parquet")

In [5]:
df_repos.show()

+--------------------+--------------------+------------+--------------------+--------------------+------+
|             file_id|           repo_name|last_updated|                path|             content|binary|
+--------------------+--------------------+------------+--------------------+--------------------+------+
|5685b691562914a36...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|
|570e460450c8a4ddf...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|
|4fcdd2682cc08d9fd...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|
|f5cfc5a75655300d6...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|
|978ff5d4a96103dd0...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|
|00f28b2e3afaa5dad...|PangPangPangPangP...|  2022-01-01|Rime/luna_pinyin_...|# Rime schema\n# ...| false|
|c1953a20a616f2875...|typesupply/extractor|  2

In [6]:
df_repos.printSchema()

root
 |-- file_id: string (nullable = true)
 |-- repo_name: string (nullable = true)
 |-- last_updated: date (nullable = true)
 |-- path: string (nullable = true)
 |-- content: string (nullable = true)
 |-- binary: boolean (nullable = true)



In [7]:
df_repos.groupBy("repo_name").count().show()

+--------------------+-----+
|           repo_name|count|
+--------------------+-----+
|    mcocdawc/chemopt|    1|
|dtaylor84/pysnapsync|    1|
|     jiasir/playback|    1|
|neherlab/HIVEVO_a...|    1|
|revegon/Sentiment...|   17|
| pgxcentre/geneparse|    2|
|mauricerkelly/dot...|    2|
|william-silversmi...|    1|
|       kolyat/vpoker|    2|
|        sloria/konch|    1|
|shaunwbell/FOCI_A...|    2|
|lakshmankumar12/v...|    1|
|unt-libraries/pyuntl|    1|
|      gerritgr/LumPy|    2|
|typesupply/extractor|    9|
|         naim94a/vix|    1|
|userlocalhost/sta...|    1|
|rachel3834/mulens...|    3|
|         Pinafore/qb|    1|
|jonasrogert/Quadc...|    1|
+--------------------+-----+
only showing top 20 rows



In [8]:
df_repos= df_repos.withColumn("year", year(col("last_updated")))
df_repos= df_repos.withColumn("month", month(col("last_updated")))
df_repos= df_repos.withColumn("date", dayofmonth(col("last_updated")))

In [9]:
df_repos.show()

+--------------------+--------------------+------------+--------------------+--------------------+------+----+-----+----+
|             file_id|           repo_name|last_updated|                path|             content|binary|year|month|date|
+--------------------+--------------------+------------+--------------------+--------------------+------+----+-----+----+
|5685b691562914a36...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|2022|    1|   1|
|570e460450c8a4ddf...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|2022|    1|   1|
|4fcdd2682cc08d9fd...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|2022|    1|   1|
|f5cfc5a75655300d6...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|2022|    1|   1|
|978ff5d4a96103dd0...|typesupply/extractor|  2022-01-01|tests/data/ibm_pl...|<?xml version='1....| false|2022|    1|   1|
|00f28b2e3afaa5dad...|Pa

In [10]:
df_repos = df_repos.drop("last_updated")

In [11]:
df_repos.write.option("header",True) \
    .partitionBy("year", "month", "date") \
    .parquet("sample_data/output", mode="overwrite")

In [12]:
!tree sample_data/output/

sample_data/output/
├── _SUCCESS
└── year=2022
    ├── month=1
    │   ├── date=1
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=10
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=11
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=12
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=13
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=14
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=15
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=16
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
    │   ├── date=17
    │   │   └── part-00001-630e597e-532f-4939-bac2-4b7e1d5cc1c5.c000.snappy.parquet
  

In [13]:
df_repos_2022_01_03 = spark.read.parquet("sample_data/output/year=2022/month=3/date=1")

In [14]:
df_repos_2022_01_03.show()

+--------------------+--------------------+--------------------+--------------------+------+
|             file_id|           repo_name|                path|             content|binary|
+--------------------+--------------------+--------------------+--------------------+------+
|f71f3d0431b8ebc8c...|    izhujiang/my_env| dotfiles/sh/.zlogin|# Finally, if the...| false|
|341f710890ec3ce53...|JeremyJStarcher/b...|bin/bash-git-prom...|# This is Jeremy'...| false|
|23fd1514d0f6a9018...|    izhujiang/my_env|dotfiles/deprecat...|" -------Java\n" ...| false|
|8c8de810a85f22e80...|       rhayes777/dmx|src/media/images/...|                null|  true|
|0abf9f39729c62f81...|rhayes777/dancest...|src/media/images/...|                null|  true|
|d9e69fba23ba8db06...|       rhayes777/dmx|src/media/audio/F...|                null|  true|
+--------------------+--------------------+--------------------+--------------------+------+



In [16]:
df_repos_2022_03_01 = spark.read.parquet("sample_data/sample_output_year=2022_month=3_date=1_part-00001-c851dc2b-76f8-4835-83b5-aa64afa4ea66.c000.snappy.parquet")

In [17]:
df_repos_2022_03_01.show()

+--------------------+--------------------+--------------------+--------------------+------+
|             file_id|           repo_name|                path|             content|binary|
+--------------------+--------------------+--------------------+--------------------+------+
|f71f3d0431b8ebc8c...|    izhujiang/my_env| dotfiles/sh/.zlogin|# Finally, if the...| false|
|341f710890ec3ce53...|JeremyJStarcher/b...|bin/bash-git-prom...|# This is Jeremy'...| false|
|23fd1514d0f6a9018...|    izhujiang/my_env|dotfiles/deprecat...|" -------Java\n" ...| false|
|8c8de810a85f22e80...|       rhayes777/dmx|src/media/images/...|                null|  true|
|0abf9f39729c62f81...|rhayes777/dancest...|src/media/images/...|                null|  true|
|d9e69fba23ba8db06...|       rhayes777/dmx|src/media/audio/F...|                null|  true|
+--------------------+--------------------+--------------------+--------------------+------+

